In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load  

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fault-data-for-sensors/data/MissingInfoData.npy
/kaggle/input/fault-data-for-sensors/data/NormalData.npy
/kaggle/input/fault-data-for-sensors/data/TrendData.npy
/kaggle/input/fault-data-for-sensors/data/MissingData.npy
/kaggle/input/fault-data-for-sensors/data/RandomData.npy
/kaggle/input/fault-data-for-sensors/data/JumpData.npy


In [2]:
# data in format 5000 x 200 x 10

import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.initializers import glorot_uniform, glorot_normal
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
np.random.seed(42)


from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from matplotlib import style
style.use('fivethirtyeight')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
import random

random_indices = random.sample(range(10001), 60)

print(random_indices)

numbers = list(range(60))

print(numbers)

In [99]:
batch_size = 50

In [100]:
random_indices = range(0,batch_size)
random_indices

range(0, 50)

In [101]:
# Checking the Data

JumpData = np.load("/kaggle/input/fault-data-for-sensors/data/JumpData.npy")[random_indices]
MissingData = np.load("/kaggle/input/fault-data-for-sensors/data/MissingData.npy")[random_indices]
NormalData = np.load("/kaggle/input/fault-data-for-sensors/data/NormalData.npy")[random_indices]
RandomData = np.load("/kaggle/input/fault-data-for-sensors/data/RandomData.npy")[random_indices]
TrendData = np.load("/kaggle/input/fault-data-for-sensors/data/TrendData.npy")[random_indices]



In [103]:
JumpData.shape, MissingData.shape, NormalData.shape, RandomData.shape, TrendData.shape

((50, 200, 10), (50, 200, 10), (50, 200, 10), (50, 200, 10), (50, 200, 10))

In [104]:
JumpLabels = np.tile([1,0,0,0,0], (10000, 1))[:batch_size] 
MissingLabels = np.tile([0,1,0,0,0], (10000, 1))[:batch_size]
NormalLabels = np.tile([0,0,1,0,0], (10000, 1))[:batch_size]
RandomLabels = np.tile([0,0,0,1,0], (10000, 1))[:batch_size]
TrendLabels = np.tile([0,0,0,0,1], (10000, 1))[:batch_size]

In [105]:
#splitting individual datas into train test val

JumpX_train, JumpX_test, Jumpy_train, Jumpy_test = train_test_split(JumpData, JumpLabels, test_size=0.2, random_state=42)
JumpX_train, JumpX_val, Jumpy_train, Jumpy_val = train_test_split(JumpX_train, Jumpy_train, test_size=0.1, random_state=42)

MissingX_train, MissingX_test, Missingy_train, Missingy_test = train_test_split(MissingData, MissingLabels, test_size=0.2, random_state=42)
MissingX_train, MissingX_val, Missingy_train, Missingy_val = train_test_split(MissingX_train, Missingy_train, test_size=0.1, random_state=42)

NormalX_train, NormalX_test, Normaly_train, Normaly_test = train_test_split(NormalData, NormalLabels, test_size=0.2, random_state=42)
NormalX_train, NormalX_val, Normaly_train, Normaly_val = train_test_split(NormalX_train, Normaly_train, test_size=0.1, random_state=42)

RandomX_train, RandomX_test, Randomy_train, Randomy_test = train_test_split(RandomData, RandomLabels, test_size=0.2, random_state=42)
RandomX_train, RandomX_val, Randomy_train, Randomy_val = train_test_split(RandomX_train, Randomy_train, test_size=0.1, random_state=42)

TrendX_train, TrendX_test, Trendy_train, Trendy_test = train_test_split(TrendData, TrendLabels, test_size=0.2, random_state=42)
TrendX_train, TrendX_val, Trendy_train, Trendy_val = train_test_split(TrendX_train, Trendy_train, test_size=0.1, random_state=42)

In [106]:
X_train = np.append(JumpX_train, MissingX_train, axis=0)
X_train = np.append(X_train, NormalX_train, axis=0)
X_train = np.append(X_train, RandomX_train, axis=0)
X_train = np.append(X_train, TrendX_train, axis=0) 

y_train = np.append(Jumpy_train, Missingy_train, axis=0)
y_train = np.append(y_train, Normaly_train, axis=0)
y_train = np.append(y_train, Randomy_train, axis=0)
y_train = np.append(y_train, Trendy_train, axis=0) 

In [107]:
X_test = np.append(JumpX_test, MissingX_test, axis=0)
X_test = np.append(X_test, NormalX_test, axis=0)
X_test = np.append(X_test, RandomX_test, axis=0)
X_test = np.append(X_test, TrendX_test, axis=0) 

y_test = np.append(Jumpy_test, Missingy_test, axis=0)
y_test = np.append(y_test, Normaly_test, axis=0)
y_test = np.append(y_test, Randomy_test, axis=0)
y_test = np.append(y_test, Trendy_test, axis=0) 

In [108]:
X_val = np.append(JumpX_val, MissingX_val, axis=0)
X_val = np.append(X_val, NormalX_val, axis=0)
X_val = np.append(X_val, RandomX_val, axis=0)
X_val = np.append(X_val, TrendX_val, axis=0) 

y_val = np.append(Jumpy_val, Missingy_val, axis=0)
y_val = np.append(y_val, Normaly_val, axis=0)
y_val = np.append(y_val, Randomy_val, axis=0)
y_val = np.append(y_val, Trendy_val, axis=0) 

In [109]:
X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape

((180, 200, 10), (50, 200, 10), (20, 200, 10), (180, 5), (50, 5), (20, 5))

In [110]:
X_train = np.round(X_train, 3)
X_test = np.round(X_test, 3)
X_val = np.round(X_val, 3)

In [111]:
# Shuffling the Data and labels

train_samples = X_train.shape[0]
test_samples = X_test.shape[0]
val_samples = X_val.shape[0]

random_indices = np.random.permutation(train_samples)
X_train = X_train[random_indices]
y_train = y_train[random_indices]

random_indices = np.random.permutation(test_samples)
X_test = X_test[random_indices]
y_test = y_test[random_indices]

random_indices = np.random.permutation(val_samples)
X_val = X_val[random_indices]
y_val = y_val[random_indices]

In [113]:
X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape

((180, 200, 10), (50, 200, 10), (20, 200, 10), (180, 5), (50, 5), (20, 5))

In [ ]:
# derive new columns

def derive_columns(data):
    
    min_vals = np.min(data, axis=1)
    max_vals = np.max(data, axis=1)
    median_vals = np.median(data, axis=1)
    avg_vals = np.mean(data, axis=1)
    std_dev_vals = np.std(data, axis=1)
    
    new_data = np.zeros((data.shape[0], 5))
    
    
    sorted_data = np.sort(data, axis=1)
    new_data[:, 0] = (max_vals - min_vals)/9
    new_data[:, 1] = (sorted_data[:, -2] - sorted_data[:, 1])/7
    new_data[:, 2] = (sorted_data[:, -3] - sorted_data[:, 2])/5
    new_data[:, 3] = (sorted_data[:, -4] - sorted_data[:, 3])/3
    new_data[:, 4] = sorted_data[:, -5] - sorted_data[:, 4]

    
    return new_data
    
    
# Test
X_train_new = np.zeros((len(X_train), 200, 5))
X_test_new = np.zeros((len(X_test), 200, 5))
X_val_new = np.zeros((len(X_val), 200, 5))
print(X_train[0][0][:])
print(derive_columns(X_train[0])[0][:])

for i in range(len(X_train)):
    X_train_new[i] = derive_columns(X_train[i])
    
for i in range(len(X_test)):
    X_test_new[i] = derive_columns(X_test[i])
    
for i in range(len(X_val)):
    X_val_new[i] = derive_columns(X_val[i])

print("done")



[ 0.553 -0.013  0.259  0.046 -4.819  0.154  0.035 -0.179  0.208  0.087]
[0.59688889 0.06257143 0.0442     0.03966667 0.041     ]
done


In [115]:
X_train_new.shape,X_test_new.shape,X_val_new.shape

((180, 200, 5), (50, 200, 5), (20, 200, 5))

In [116]:
X_train= X_train_new
X_test= X_test_new
X_val= X_val_new

# Initial stage : CNN MODEL FOR FAULT DETECTION AND CLASSIFICATION

In [ ]:
classifier_model = keras.models.Sequential([    
    keras.layers.Conv2D(filters=10, kernel_size=(11,1),kernel_initializer=glorot_normal() ,activation='tanh',padding='same' ,input_shape=(200,5,1)),
    keras.layers.MaxPool2D(pool_size=(2, 1)),

    
    keras.layers.Conv2D(filters=20, kernel_size=(5,1),kernel_initializer=glorot_normal() ,activation='tanh' ,padding='same'), 
    keras.layers.MaxPool2D(pool_size=(2, 1)),
    keras.layers.Dropout(rate=0.5),
    
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation='tanh',kernel_initializer=glorot_normal()), # glorot_uniform
    keras.layers.Dense(5, activation='softmax')
])

In [ ]:
from timeit import default_timer as timer

class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

cb = TimingCallback()

In [ ]:
classifier_model.summary()

In [ ]:
opt = Adam(lr=lr)
classifier_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:

lr = 0.0002
epochs = 200

In [ ]:

hist = classifier_model.fit(X_train, y_train, epochs=epochs,batch_size=1000 ,validation_data= (X_val, y_val), callbacks=[cb])

In [ ]:
print(cb.logs)
print(sum(cb.logs))

In [ ]:
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs = range(1, len(train_loss) + 1)


plt.figure(figsize=(10, 5))
plt.plot(epochs, train_loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
train_loss[-1], val_loss[-1]

In [ ]:
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']


epochs = range(1, len(train_loss) + 1)


plt.figure(figsize=(10, 5))
plt.plot(epochs, train_acc, 'bo', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
train_acc[-1], val_acc[-1]

In [ ]:

y_pred = classifier_model.predict(X_test)

y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_true_labels, y_pred_labels)

cm = cm/2000


disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Spike", "Missing", "Normal", "Random", "Drift"])

disp.plot(cmap='Blues')
plt.grid(False)
plt.show()

In [ ]:
tp = np.diag(cm)
fp = np.sum(cm, axis=0) - tp
fn = np.sum(cm, axis=1) - tp
tn = np.sum(cm) - (tp + fp + fn)

tp = np.sum(tp)
fp = np.sum(fp)
fn = np.sum(fn)
tn = np.sum(tn)
tp, fp, fn, tn

In [ ]:
precision = precision_score(y_true_labels, y_pred_labels, average='weighted')
recall = recall_score(y_true_labels, y_pred_labels, average='weighted')
accuracy = accuracy_score(y_true_labels, y_pred_labels)
f1 = f1_score(y_true_labels, y_pred_labels, average='weighted')

precision, recall, accuracy, f1

In [ ]:
import time
from timeit import default_timer as timer

class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

cb = TimingCallback()

batch_size = 50
lr = 0.0002
epochs = 200

all_metrics = []

for i in range(10):
    print(f"------------------- TRY {i+1} ----------------------------")
    classifier_model = keras.models.Sequential([    
        keras.layers.Conv2D(filters=10, kernel_size=(11,1),kernel_initializer=glorot_normal() ,activation='tanh',padding='same' ,input_shape=(200,5,1)),
        keras.layers.MaxPool2D(pool_size=(2, 1)),


        keras.layers.Conv2D(filters=20, kernel_size=(5,1),kernel_initializer=glorot_normal() ,activation='tanh' ,padding='same'), 
        keras.layers.MaxPool2D(pool_size=(2, 1)),
        keras.layers.Dropout(rate=0.5),

        keras.layers.Flatten(),
        keras.layers.Dense(100, activation='tanh',kernel_initializer=glorot_normal()), # glorot_uniform
        keras.layers.Dense(5, activation='softmax')
    ])


    opt = Adam(lr=lr)
    classifier_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    cb = TimingCallback()

 
    hist = classifier_model.fit(X_train, y_train, epochs=epochs,batch_size=1000 ,validation_data= (X_val, y_val), callbacks=[cb])
    

    train_loss = hist.history['loss']
    val_loss = hist.history['val_loss']


    epch = range(1, len(train_loss) + 1)


    plt.plot(epch, train_loss, 'b', label='Training Loss')
    plt.plot(epch, val_loss, 'r', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.tight_layout()
    plt.savefig(f'loss_plot_{batch_size}_{i}.png')
    plt.close()



    train_acc = hist.history['accuracy']
    val_acc = hist.history['val_accuracy']


    epch = range(1, len(train_loss) + 1)


    plt.plot(epch, train_acc, 'b', label='Training Accuracy')
    plt.plot(epch, val_acc, 'r', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.tight_layout()
    plt.savefig(f'accuracy_plot_{batch_size}_{i}.png')
    plt.close()

    classifier_model.save(f'model_{batch_size}_{i}.h5')
 
    start_test_time = time.time()
    y_pred = classifier_model.predict(X_test)
    end_test_time = time.time()
    testing_time = end_test_time - start_test_time

    y_pred_labels = np.argmax(y_pred, axis=1)
    y_true_labels = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_true_labels, y_pred_labels)


    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    tn = np.sum(cm) - (tp + fp + fn)

    tp = np.sum(tp)
    fp = np.sum(fp)
    fn = np.sum(fn)
    tn = np.sum(tn)

    precision = precision_score(y_true_labels, y_pred_labels, average='weighted')
    recall = recall_score(y_true_labels, y_pred_labels, average='weighted')
    test_accuracy = accuracy_score(y_true_labels, y_pred_labels)
    f1 = f1_score(y_true_labels, y_pred_labels, average='weighted')


    metrics = {
            'Train Accuracy' :max(train_acc),
            'Validation Accuracy' : max(val_acc),
            'Test Accuracy': test_accuracy,
            'TP': tp,
            'TN': tn,
            'FP': fp,
            'FN': fn,
            'Precision': precision,
            'Recall': recall,
            'F1': f1,
            'Training Time': sum(cb.logs),
            'Testing Time' : testing_time
        }

    all_metrics.append(metrics)

    with open(f'results_{batch_size}_{i}.txt', 'w') as f:
            for key, value in metrics.items():
                f.write(f'{key}: {value}\n')

                

all_train_accuracy = []
all_val_accuracy = []
all_test_accuracy = []
all_tp = []
all_tn = []
all_fp = []
all_fn = []
all_precision = []
all_recall = []
all_f1 = []
all_training_time = []
all_testing_time = []


for metrics in all_metrics:
    all_train_accuracy.append(metrics['Train Accuracy'])
    all_val_accuracy.append(metrics['Validation Accuracy'])
    all_test_accuracy.append(metrics['Test Accuracy'])
    all_tp.append(metrics['TP'])
    all_tn.append(metrics['TN'])
    all_fp.append(metrics['FP'])
    all_fn.append(metrics['FN'])
    all_precision.append(metrics['Precision'])
    all_recall.append(metrics['Recall'])
    all_f1.append(metrics['F1'])
    all_training_time.append(metrics['Training Time'])
    all_testing_time.append(metrics['Testing Time'])


average_metrics = {
    'Train Accuracy': np.mean(all_train_accuracy),
    'Validation Accuracy': np.mean(all_val_accuracy),
    'Test Accuracy': np.mean(all_test_accuracy),
    'TP': np.mean(all_tp),
    'TN': np.mean(all_tn),
    'FP': np.mean(all_fp),
    'FN': np.mean(all_fn),
    'Precision': np.mean(all_precision),
    'Recall': np.mean(all_recall),
    'F1': np.mean(all_f1),
    'Training Time': np.mean(all_training_time),
    'Testing Time' : np.mean(all_testing_time)
}

with open(f'results_{batch_size}_avg.txt', 'w') as f:
            for key, value in average_metrics.items():
                f.write(f'{key}: {value}\n')

------------------- TRY 1 ----------------------------
Epoch 1/200


2024-04-20 15:32:17.320558: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_51/dropout_51/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1/1 [==============================] - 2s 2s/step - loss: 1.6198 - accuracy: 0.1333 - val_loss: 1.5451 - val_accuracy: 0.2000
Epoch 2/200
1/1 [==============================] - 0s 32ms/step - loss: 1.5649 - accuracy: 0.2000 - val_loss: 1.4722 - val_accuracy: 0.2000
Epoch 3/200
1/1 [==============================] - 0s 34ms/step - loss: 1.4991 - accuracy: 0.2111 - val_loss: 1.4108 - val_accuracy: 0.3000
Epoch 4/200
1/1 [==============================] - 0s 32ms/step - loss: 1.4407 - accuracy: 0.3778 - val_loss: 1.3629 - val_accuracy: 0.3000
Epoch 5/200
1/1 [==============================] - 0s 32ms/step - loss: 1.4021 - accuracy: 0.3389 - val_loss: 1.3245 - val_accuracy: 0.4000
Epoch 6/200
1/1 [==============================] - 0s 30ms/step - loss: 1.3630 - accuracy: 0.4667 - val_loss: 1.2944 - val_accuracy: 0.5000
Epoch 7/200
1/1 [==============================] - 0s 32ms/step - loss: 1.3228 - accuracy: 0.5944 - val_loss: 1.2700 - val_accuracy: 0.5000
Epoch 8/200
1/1 [=================

2024-04-20 15:32:27.091122: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_52/dropout_52/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1/1 [==============================] - 2s 2s/step - loss: 1.6082 - accuracy: 0.2111 - val_loss: 1.5328 - val_accuracy: 0.2000
Epoch 2/200
1/1 [==============================] - 0s 31ms/step - loss: 1.5616 - accuracy: 0.2000 - val_loss: 1.4648 - val_accuracy: 0.2000
Epoch 3/200
1/1 [==============================] - 0s 33ms/step - loss: 1.4983 - accuracy: 0.2611 - val_loss: 1.4011 - val_accuracy: 0.4500
Epoch 4/200
1/1 [==============================] - 0s 33ms/step - loss: 1.4429 - accuracy: 0.4333 - val_loss: 1.3469 - val_accuracy: 0.4500
Epoch 5/200
1/1 [==============================] - 0s 32ms/step - loss: 1.3934 - accuracy: 0.4722 - val_loss: 1.3006 - val_accuracy: 0.5000
Epoch 6/200
1/1 [==============================] - 0s 33ms/step - loss: 1.3433 - accuracy: 0.5611 - val_loss: 1.2596 - val_accuracy: 0.6500
Epoch 7/200
1/1 [==============================] - 0s 33ms/step - loss: 1.3070 - accuracy: 0.5778 - val_loss: 1.2235 - val_accuracy: 0.5500
Epoch 8/200
1/1 [=================

...

1/1 [==============================] - 2s 2s/step - loss: 1.6082 - accuracy: 0.2000 - val_loss: 1.5435 - val_accuracy: 0.2000
Epoch 2/200
1/1 [==============================] - 0s 39ms/step - loss: 1.5625 - accuracy: 0.2111 - val_loss: 1.4854 - val_accuracy: 0.2000
Epoch 3/200
1/1 [==============================] - 0s 38ms/step - loss: 1.5085 - accuracy: 0.2667 - val_loss: 1.4241 - val_accuracy: 0.4000
Epoch 4/200
1/1 [==============================] - 0s 37ms/step - loss: 1.4601 - accuracy: 0.4056 - val_loss: 1.3669 - val_accuracy: 0.4500
Epoch 5/200
1/1 [==============================] - 0s 35ms/step - loss: 1.4058 - accuracy: 0.5833 - val_loss: 1.3148 - val_accuracy: 0.4000
Epoch 6/200
1/1 [==============================] - 0s 36ms/step - loss: 1.3642 - accuracy: 0.5833 - val_loss: 1.2669 - val_accuracy: 0.5000
Epoch 7/200
1/1 [==============================] - 0s 46ms/step - loss: 1.3118 - accuracy: 0.6889 - val_loss: 1.2226 - val_accuracy: 0.5500
Epoch 8/200
1/1 [=================

2024-04-20 15:33:35.688468: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_59/dropout_59/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1/1 [==============================] - 2s 2s/step - loss: 1.6141 - accuracy: 0.1722 - val_loss: 1.5518 - val_accuracy: 0.2000
Epoch 2/200
1/1 [==============================] - 0s 36ms/step - loss: 1.5651 - accuracy: 0.2000 - val_loss: 1.4787 - val_accuracy: 0.2500
Epoch 3/200
1/1 [==============================] - 0s 33ms/step - loss: 1.4986 - accuracy: 0.3111 - val_loss: 1.4154 - val_accuracy: 0.3000
Epoch 4/200
1/1 [==============================] - 0s 35ms/step - loss: 1.4376 - accuracy: 0.4056 - val_loss: 1.3640 - val_accuracy: 0.4500
Epoch 5/200
1/1 [==============================] - 0s 31ms/step - loss: 1.3957 - accuracy: 0.4500 - val_loss: 1.3224 - val_accuracy: 0.4000
Epoch 6/200
1/1 [==============================] - 0s 32ms/step - loss: 1.3531 - accuracy: 0.5333 - val_loss: 1.2877 - val_accuracy: 0.4000
Epoch 7/200
1/1 [==============================] - 0s 32ms/step - loss: 1.3147 - accuracy: 0.5667 - val_loss: 1.2568 - val_accuracy: 0.5000
Epoch 8/200
1/1 [=================

2024-04-20 15:33:45.452775: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_60/dropout_60/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1/1 [==============================] - 2s 2s/step - loss: 1.5973 - accuracy: 0.2278 - val_loss: 1.5016 - val_accuracy: 0.2000
Epoch 2/200
1/1 [==============================] - 0s 32ms/step - loss: 1.5222 - accuracy: 0.2444 - val_loss: 1.4255 - val_accuracy: 0.4000
Epoch 3/200
1/1 [==============================] - 0s 30ms/step - loss: 1.4589 - accuracy: 0.3667 - val_loss: 1.3590 - val_accuracy: 0.3500
Epoch 4/200
1/1 [==============================] - 0s 31ms/step - loss: 1.3908 - accuracy: 0.5000 - val_loss: 1.3045 - val_accuracy: 0.3500
Epoch 5/200
1/1 [==============================] - 0s 32ms/step - loss: 1.3500 - accuracy: 0.4389 - val_loss: 1.2619 - val_accuracy: 0.5000
Epoch 6/200
1/1 [==============================] - 0s 30ms/step - loss: 1.2999 - accuracy: 0.5611 - val_loss: 1.2272 - val_accuracy: 0.5500
Epoch 7/200
1/1 [==============================] - 0s 32ms/step - loss: 1.2709 - accuracy: 0.5500 - val_loss: 1.1988 - val_accuracy: 0.4500
Epoch 8/200
1/1 [=================

---------------------------------------------------------

In [ ]:
classifier_model = keras.models.Sequential([    
    keras.layers.Conv2D(filters=10, kernel_size=(11,1),kernel_initializer=glorot_normal() ,activation='tanh',padding='same' ,input_shape=(200,10,1)),
    keras.layers.MaxPool2D(pool_size=(2, 1)),

    
    keras.layers.Conv2D(filters=20, kernel_size=(5,1),kernel_initializer=glorot_normal() ,activation='tanh' ,padding='same'), 
    keras.layers.MaxPool2D(pool_size=(2, 1)),
    keras.layers.Dropout(rate=0.5),
    
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation='tanh',kernel_initializer=glorot_normal()), # glorot_uniform
    keras.layers.Dense(5, activation='softmax')
])

In [ ]:
lr = 0.0002
epochs = 200

opt = Adam(lr=lr)
classifier_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:

hist = classifier_model.fit(X_train, y_train, epochs=epochs,batch_size=1000 ,validation_data= (X_val, y_val))


In [ ]:
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']


epochs = range(1, len(train_loss) + 1)

plt.figure(figsize=(10, 5))
plt.plot(epochs, train_loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()  

In [ ]:

y_pred = classifier_model.predict(X_test)

y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

confusion = confusion_matrix(y_true_labels, y_pred_labels)

confusion = confusion/2000


disp = ConfusionMatrixDisplay(confusion_matrix=confusion, display_labels=["Spike", "Missing", "Normal", "Random", "Drift"])

disp.plot(cmap='Blues')
plt.grid(False)
plt.show()

---------------------------------------------------------------------

In [ ]:
classifier_model = keras.models.Sequential([    
    keras.layers.Conv2D(filters=10, kernel_size=(11,1),kernel_initializer=glorot_normal() ,activation='tanh',padding='same' ,input_shape=(200,10,1)),
    keras.layers.MaxPool2D(pool_size=(2, 1)),

    
    keras.layers.Conv2D(filters=20, kernel_size=(5,1),kernel_initializer=glorot_normal() ,activation='tanh' ,padding='same'), 
    keras.layers.MaxPool2D(pool_size=(2, 1)),
    keras.layers.Dropout(rate=0.5),
    
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation='tanh',kernel_initializer=glorot_normal()), # glorot_uniform
    keras.layers.Dense(5, activation='softmax')
])

lr = 0.0002
epochs = 200

opt = Adam(lr=lr)
classifier_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


hist = classifier_model.fit(X_train, y_train, epochs=epochs,batch_size=1000 ,validation_data= (X_val, y_val))


In [ ]:
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']


epochs = range(1, len(train_loss) + 1)


plt.figure(figsize=(10, 5))
plt.plot(epochs, train_loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()  

In [ ]:

y_pred = classifier_model.predict(X_test)

y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

confusion = confusion_matrix(y_true_labels, y_pred_labels)

confusion = confusion/2000


disp = ConfusionMatrixDisplay(confusion_matrix=confusion, display_labels=["Spike", "Missing", "Normal", "Random", "Drift"])

disp.plot(cmap='Blues')
plt.grid(False)
plt.show()

---------------------------------------------------------------------------------------------

In [ ]:
classifier_model = keras.models.Sequential([    
    keras.layers.Conv2D(filters=10, kernel_size=(11,1),kernel_initializer=glorot_normal() ,activation='tanh',padding='same' ,input_shape=(200,10,1)),
    keras.layers.MaxPool2D(pool_size=(2, 1)),

    
    keras.layers.Conv2D(filters=20, kernel_size=(5,1),kernel_initializer=glorot_normal() ,activation='tanh' ,padding='same'), 
    keras.layers.MaxPool2D(pool_size=(2, 1)),
    keras.layers.Dropout(rate=0.5),
    
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation='tanh',kernel_initializer=glorot_normal()), # glorot_uniform
    keras.layers.Dense(5, activation='softmax')
])

lr = 0.0002
epochs = 200

opt = Adam(lr=lr)
classifier_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


hist = classifier_model.fit(X_train, y_train, epochs=epochs,batch_size=1000 ,validation_data= (X_val, y_val))


In [ ]:
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']


epochs = range(1, len(train_loss) + 1)

plt.figure(figsize=(10, 5))
plt.plot(epochs, train_loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()  

# Reconstruciton Stage: Auto Encoders


In [ ]:
# First For Jump Data

X_jump_train, X_jump_test, y_normal_train, y_normal_test = train_test_split(JumpData, NormalData, test_size=0.2, random_state=42)
X_jump_train, X_jump_val, y_normal_train, y_normal_val = train_test_split(X_jump_train, y_normal_train, test_size=0.1, random_state=42)

In [ ]:
X_jump_train.shape, X_jump_test.shape, X_jump_val.shape, y_normal_train.shape, y_normal_test.shape, y_normal_val.shape

In [ ]:
y_normal_train[1,1,:]

In [ ]:
reconstructor_model = keras.models.Sequential([    
    # Encoder
    keras.layers.Conv2D(filters=10, kernel_size=(31,5), activation='relu',kernel_initializer=glorot_uniform(3),padding='same' ,input_shape=(200,10,1)),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    
    
    keras.layers.Conv2D(filters=20, kernel_size=(31,3), activation='relu',kernel_initializer=glorot_uniform(3) ,padding='same'), 
    keras.layers.MaxPool2D(pool_size=(2, 1)),
    
    keras.layers.Flatten(), # 5000
    keras.layers.Dense(10000, activation='relu',kernel_initializer=glorot_uniform(3)), # 5000
    keras.layers.Dense(500, activation='relu',kernel_initializer=glorot_uniform(3)),
    
    
    # Decoder
    keras.layers.Dense(500, activation='relu',kernel_initializer=glorot_uniform(3)),
#     keras.layers.Dense(10000, activation='relu',kernel_initializer=glorot_uniform(3)),
    keras.layers.Dense(5000, activation='relu',kernel_initializer=glorot_uniform(3)), # Doubtful, ask 
    keras.layers.Reshape((50,5,20)),
    
    keras.layers.Conv2D(filters=20, kernel_size=(31,3), activation='relu' ,padding='same',kernel_initializer=glorot_uniform(3)),
    keras.layers.UpSampling2D(size=(2, 1)),
    
    keras.layers.Conv2D(filters=10, kernel_size=(31,5), activation='relu' ,padding='same',kernel_initializer=glorot_uniform(3)),
    keras.layers.UpSampling2D(size=(2, 2)),
    
    keras.layers.Conv2D(filters=1, kernel_size=(31,5), activation='relu' ,padding='same',kernel_initializer=glorot_uniform(3)),  
])  

In [ ]:
lr = 0.0001
epochs = 200

opt = Adam(lr=lr)
reconstructor_model.compile(loss='mse', optimizer=opt)

In [ ]:
reconstructor_model.summary()

In [ ]:

epochs = 20           
jump_rec_hist = reconstructor_model.fit(X_jump_train, y_normal_train, epochs=epochs,batch_size=50 ,validation_data= (X_jump_val, y_normal_val))

In [ ]:
train_loss = jump_rec_hist.history['loss']
val_loss = jump_rec_hist.history['val_loss']

epochs = range(1, len(train_loss) + 1)


plt.figure(figsize=(10, 5))
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss,  label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
y_pred = reconstructor_model.predict(X_jump_test)

In [ ]:
y_pred.shape

In [ ]:
y_pred[1,:,:,0].shape

In [ ]:
y_normal_test[1].shape

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(y_normal_test[1,:,1], label='Real Value')
plt.plot(y_pred[1,:,1,0], label='Predicted Value')
plt.xlabel('Data Points')
plt.ylabel('Value')
plt.legend()
plt.title('Real vs. Predicted Value')
plt.show()